<a id='top'></a>
<h1 style="text-align:center;font-size:200%;;">Real or Not? NLP with Disaster Tweets</h1>
<img src="https://dataxboost.files.wordpress.com/2018/03/nlp.jpg">

## Competition Description
* Twitter has become an important communication channel in times of emergency.
The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Notebook Content!</h3>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#libraries" role="tab" aria-controls="profile">Import Libraries<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#load" role="tab" aria-controls="messages">Load Data<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action"  data-toggle="list" href="#visual" role="tab" aria-controls="settings">Visualization of data<span class="badge badge-primary badge-pill">3</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#word" role="tab" aria-controls="settings">WordCloud<span class="badge badge-primary badge-pill">4</span></a> 
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#clean" role="tab" aria-controls="settings">Cleaning the text<span class="badge badge-primary badge-pill">5</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#split" role="tab" aria-controls="settings">Train and test Split<span class="badge badge-primary badge-pill">6</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#model" role="tab" aria-controls="settings"> Creating the Model<span class="badge badge-primary badge-pill">7</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#eval" role="tab" aria-controls="settings">Model Evaluation<span class="badge badge-primary badge-pill">8</span></a>  

<a id='libraries'></a>
## 1. Import Libraries

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time 
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import plotly.graph_objects as go
import re


# Natural Language Tool Kit 
import nltk  
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from collections import Counter
import cufflinks as cf
cf.go_offline()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id='load'></a>
# 2. Load Data

In [7]:
# train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
# train = pd.read_csv("https://www.kaggle.com/c/nlp-getting-started/download/jkgYvkO3wssyod4THUn8%2Fversions%2FzgYOOcJ9rQDp8lZbDK1b%2Ffiles%2Ftrain.csv")

!wget https://www.kaggle.com/c/nlp-getting-started/download/jkgYvkO3wssyod4THUn8%2Fversions%2FzgYOOcJ9rQDp8lZbDK1b%2Ffiles%2Ftrain.csv
fname = "train.csv"

fid = open(fname, 'r')
train=pd.read_csv(fname,sep='|')
fid.close()


# test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
# test = pd.read_csv("https://www.kaggle.com/c/nlp-getting-started/download/jkgYvkO3wssyod4THUn8%2Fversions%2FzgYOOcJ9rQDp8lZbDK1b%2Ffiles%2Ftest.csv")

!wget https://www.kaggle.com/c/nlp-getting-started/download/jkgYvkO3wssyod4THUn8%2Fversions%2FzgYOOcJ9rQDp8lZbDK1b%2Ffiles%2Ftest.csv
fname = "test.csv"

fid = open(fname, 'r')
test=pd.read_csv(fname,sep='|')
fid.close()

# submission =  pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
# submission =  pd.read_csv("https://www.kaggle.com/c/nlp-getting-started/download/jkgYvkO3wssyod4THUn8%2Fversions%2FzgYOOcJ9rQDp8lZbDK1b%2Ffiles%2Fsample_submission.csv")

!wget https://www.kaggle.com/c/nlp-getting-started/download/jkgYvkO3wssyod4THUn8%2Fversions%2FzgYOOcJ9rQDp8lZbDK1b%2Ffiles%2Fsample_submission.csv
fname = "sample_submission.csv"

fid = open(fname, 'r')
submission=pd.read_csv(fname,sep='|')
fid.close()

--2019-12-30 08:54:05--  https://www.kaggle.com/c/nlp-getting-started/download/jkgYvkO3wssyod4THUn8%2Fversions%2FzgYOOcJ9rQDp8lZbDK1b%2Ffiles%2Ftrain.csv
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.kaggle.com/account/login?ReturnUrl=%2Fc%2Fnlp-getting-started%2Fdownload%2FjkgYvkO3wssyod4THUn8%252Fversions%252FzgYOOcJ9rQDp8lZbDK1b%252Ffiles%252Ftrain.csv [following]
--2019-12-30 08:54:05--  https://www.kaggle.com/account/login?ReturnUrl=%2Fc%2Fnlp-getting-started%2Fdownload%2FjkgYvkO3wssyod4THUn8%252Fversions%252FzgYOOcJ9rQDp8lZbDK1b%252Ffiles%252Ftrain.csv
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘jkgYvkO3wssyod4THUn8%2Fversions%2FzgYOOcJ9rQDp8lZbDK1b%2Ffiles%2Ftrain.csv.1’

jkgYvkO3wssyod4THUn     [ <=>                ]   8

FileNotFoundError: ignored

In [0]:
train.head()

<ul style="list-style-type:square;">
  <li><span class="label label-default">id</span> a unique identifier for each tweet</li>
  <li><span class="label label-default">text </span> the text of the tweet</li>
  <li><span class="label label-default">location</span>  the location the tweet was sent from (may be blank)</li>
    <li><span class="label label-default">keyword</span>  a particular keyword from the tweet (may be blank)</li>
    <li><span class="label label-default">target</span>  in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)</li>
</ul>


In [0]:
test.head()

In [0]:
display(HTML(f"""
   
        <ul class="list-group">
          <li class="list-group-item disabled" aria-disabled="true"><h4>Shape of Train and Test Dataset</h4></li>
          <li class="list-group-item"><h4>Number of rows in Train dataset is: <span class="label label-primary">{ train.shape[0]:,}</span></h4></li>
          <li class="list-group-item"> <h4>Number of columns Train dataset is <span class="label label-primary">{train.shape[1]}</span></h4></li>
          <li class="list-group-item"><h4>Number of rows in Test dataset is: <span class="label label-success">{ test.shape[0]:,}</span></h4></li>
          <li class="list-group-item"><h4>Number of columns Test dataset is <span class="label label-success">{test.shape[1]}</span></h4></li>
        </ul>
  
    """))

In [0]:
train.info()

<a id='visual'></a>
# 3. Visualization of data

In [0]:
missing = train.isnull().sum()  
missing[missing>0].sort_values(ascending=False).iplot(kind='bar',title='Null values present in train Dataset', color=['red'])


In [0]:
train.target.value_counts().iplot(kind='bar',text=['Fake', 'Real'], title='Comparing Tweet is a real disaster (1) or not (0)',color=['blue'])

In [0]:
counts_train = train.target.value_counts(sort=False)
labels = counts_train.index
values_train = counts_train.values

data = go.Pie(labels=labels, values=values_train ,pull=[0.03, 0])
layout = go.Layout(title='Comparing Tweet is a real disaster (1) or not (0) in %')

fig = go.Figure(data=[data], layout=layout)
fig.update_traces(hole=.3, hoverinfo="label+percent+value")
fig.update_layout(
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Train', x=0.5, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [0]:
train['length'] = train['text'].apply(len)

In [0]:
data = [
    go.Box(
        y=train[train['target']==0]['length'],
        name='Fake'
    ),
    go.Box(
        y=train[train['target']==1]['length'],
        name='Real'
    )
]
layout = go.Layout(
    title = 'Comparison of text length in Tweets '
)
fig = go.Figure(data=data, layout=layout)
fig.show()

In [0]:
print('The max len of text',len(train.text.max()))
print('The min len of text is',len(train.text.min()))


In [0]:
train.keyword.nunique()  # Total of 221 unique keywords

In [0]:

train.keyword.value_counts()[:20].iplot(kind='bar', title='Top 20 keywords in text', color='red')

In [0]:
train.location.value_counts()[:20].iplot(kind='bar', title='Top 20 location in tweet', color='blue')  # Check the top 15 locations 

 <a id='word'></a>
#  4. WordCloud

In [0]:
STOPWORDS.add('https')  # remove htps to the world Cloud

def Plot_world(text):
    
    comment_words = ' '
    stopwords = set(STOPWORDS) 
    
    for val in text: 

        # typecaste each val to string 
        val = str(val) 

        # split the value 
        tokens = val.split() 

        # Converts each token into lowercase 
        for i in range(len(tokens)): 
            tokens[i] = tokens[i].lower() 

        for words in tokens: 
            comment_words = comment_words + words + ' '


    wordcloud = WordCloud(width = 5000, height = 4000, 
                    background_color ='black', 
                    stopwords = stopwords, 
                    min_font_size = 10).generate(comment_words) 

    # plot the WordCloud image                        
    plt.figure(figsize = (12, 12), facecolor = 'k', edgecolor = 'k' ) 
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 

    plt.show() 

In [0]:
text = train.text.values

Plot_world(text)


<a id='clean'></a>
# 5. Cleaning the text

In [0]:
#How many http words has this text?
train.loc[train['text'].str.contains('http')].target.value_counts()

In [0]:
pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

def remove_html(text):
    no_html= pattern.sub('',text)
    return no_html

In [0]:
# Remove all text that start with html
train['text']=train['text'].apply(lambda x : remove_html(x))

In [0]:
# lets check if this clean works
train.loc[train['text'].str.contains('http')].target.value_counts()

In [0]:
# Remove all text that start with html in test
test['text']=test['text'].apply(lambda x : remove_html(x))

### Now remove stopwords, pass to lower add delimiter and more

In [0]:
def clean_text(text):
 
    text = re.sub('[^a-zA-Z]', ' ', text)  

    text = text.lower()  

    # split to array(default delimiter is " ") 
    text = text.split()  
    
    text = [w for w in text if not w in set(stopwords.words('english'))] 

    text = ' '.join(text)    
            
    return text

In [0]:
text = train.text[3]
print(text)
clean_text(text)

In [0]:
# Apply clean text 
train['text'] = train['text'].apply(lambda x : clean_text(x))

In [0]:
# Apply clean text 
test['text']=test['text'].apply(lambda x : clean_text(x))

In [0]:
# How many unique words have this text
def counter_word (text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [0]:
text_values = train["text"]

counter = counter_word(text_values)

In [0]:
print(f"The len of unique words is: {len(counter)}")
list(counter.items())[:10]

<a id='split'></a>
# 6. Train Test Split

In [0]:
# The maximum number of words to be used. (most frequent)

vocab_size = len(counter)
embedding_dim = 32

# Max number of words in each complaint.
max_length = 20
trunc_type='post'
padding_type='post'

# oov_took its set for words out our word index
oov_tok = "<XXX>"
training_size = 6090
seq_len = 12

In [0]:
# this is base in 80% of the data, an only text and targert at this moment

training_sentences = train.text[0:training_size]
training_labels = train.target[0:training_size]

testing_sentences = train.text[training_size:]
testing_labels = train.target[training_size:]

In [0]:

print('The Shape of training ',training_sentences.shape)
print('The Shape of testing',testing_sentences.shape)


In [0]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)


In [0]:
word_index = tokenizer.word_index

In [0]:
# Lets see the first 10 elements
print("THe first word Index are: ")
for x in list(word_index)[0:15]:
    print (" {},  {} ".format(x,  word_index[x]))

# If you want to see completed -> word_index

In [0]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [0]:
print(train.text[1])
print(training_sequences[1])

## check Inverse for see how it works

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


In [0]:
# Lets see the first 10 elements
print("THe first reverse word Index are: ")
for x in list(reverse_word_index)[0:15]:
    print (" {},  {} ".format(x,  reverse_word_index[x]))

# If you want to see completed -> reverse_word_index

In [0]:
def decode(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [0]:
decode(training_sequences[1]) # this can be usefull for check predictions

In [0]:
training_padded[1628]

In [0]:

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)



<a id='model'></a>
# 7. Creating the Model

    # For a binary classification problem
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
                                    

In [0]:
# Model Definition with LSTM

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(14, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # remember this is a binary clasification
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [0]:
model.summary()


In [0]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [0]:

start_time = time.time()

num_epochs = 10
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))

final_time = (time.time()- start_time)/60
print(f'The time in minutos: {final_time}')


In [0]:
model_loss = pd.DataFrame(model.history.history)
model_loss.head()

In [0]:
model_loss[['accuracy','val_accuracy']].plot(ylim=[0,1]);

<a id='eval'></a>
# 8. Model Evaluation

In [0]:
predictions = model.predict_classes(testing_padded)   # predict_ clases because is classification problem with the split test

In [0]:
predictions

In [0]:
from sklearn.metrics import classification_report,confusion_matrix

In [0]:
# Showing Confusion Matrix
def plot_cm(y_true, y_pred, title, figsize=(5,4)):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    plt.title(title)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)

In [0]:
# Showing Confusion Matrix
plot_cm(testing_labels,predictions, 'Confution matrix of Tweets', figsize=(7,7))

# Now working with test dataset

In [0]:

testing_sequences2 = tokenizer.texts_to_sequences(test.text)
testing_padded2 = pad_sequences(testing_sequences2, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [0]:
predictions = model.predict(testing_padded2)

In [0]:
# sample of submission
submission.head()

In [0]:
submission['target'] = (predictions > 0.5).astype(int)

In [0]:
submission

In [0]:
submission.to_csv("submission.csv", index=False, header=True)

<h2>I hope this notebook <span style="color:red">Usefull</span> for you! </h3>

<a href="#top" class="btn btn-primary btn-lg active" role="button" aria-pressed="true">Go to TOP</a>
